In [1]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "execution_count": 2,
      "metadata": {
        "id": "KKBEvZqx4vvY"
      },
      "outputs": [],
      "source": [
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "from sklearn.metrics.pairwise import cosine_similarity\n",
        "from sklearn.feature_extraction.text import TfidfVectorizer\n",
        "from scipy.sparse.linalg import svds"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "\n",
        "!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip\n",
        "!unzip -n ml-latest-small.zip\n",
        "\n",
        "\n",
        "movies = pd.read_csv('/content/ml-latest-small/movies.csv')\n",
        "ratings = pd.read_csv('/content/ml-latest-small/ratings.csv')\n",
        "\n",
        "\n",
        "print(movies.head())\n",
        "print(ratings.head())\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "x-kQfmRa5Zj4",
        "outputId": "e81489ea-d787-463a-e0aa-ff055da183f9"
      },
      "execution_count": 5,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "File ‘ml-latest-small.zip’ already there; not retrieving.\n",
            "\n",
            "Archive:  ml-latest-small.zip\n",
            "   movieId                               title  \\\n",
            "0        1                    Toy Story (1995)   \n",
            "1        2                      Jumanji (1995)   \n",
            "2        3             Grumpier Old Men (1995)   \n",
            "3        4            Waiting to Exhale (1995)   \n",
            "4        5  Father of the Bride Part II (1995)   \n",
            "\n",
            "                                        genres  \n",
            "0  Adventure|Animation|Children|Comedy|Fantasy  \n",
            "1                   Adventure|Children|Fantasy  \n",
            "2                               Comedy|Romance  \n",
            "3                         Comedy|Drama|Romance  \n",
            "4                                       Comedy  \n",
            "   userId  movieId  rating  timestamp\n",
            "0       1        1     4.0  964982703\n",
            "1       1        3     4.0  964981247\n",
            "2       1        6     4.0  964982224\n",
            "3       1       47     5.0  964983815\n",
            "4       1       50     5.0  964982931\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "\n",
        "data = pd.merge(ratings, movies, on='movieId')\n",
        "\n",
        "\n",
        "print(data.head())\n",
        "\n",
        "\n",
        "pivot_table = data.pivot_table(index='userId', columns='title', values='rating')\n",
        "\n",
        "\n",
        "pivot_table.fillna(0, inplace=True)\n",
        "\n",
        "\n",
        "pivot_table.head()\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 643
        },
        "id": "Vc247QVd6Jpt",
        "outputId": "de4b2788-3156-42cc-dee6-19170e0960f8"
      },
      "execution_count": 7,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "   userId  movieId  rating   timestamp             title  \\\n",
            "0       1        1     4.0   964982703  Toy Story (1995)   \n",
            "1       5        1     4.0   847434962  Toy Story (1995)   \n",
            "2       7        1     4.5  1106635946  Toy Story (1995)   \n",
            "3      15        1     2.5  1510577970  Toy Story (1995)   \n",
            "4      17        1     4.5  1305696483  Toy Story (1995)   \n",
            "\n",
            "                                        genres  \n",
            "0  Adventure|Animation|Children|Comedy|Fantasy  \n",
            "1  Adventure|Animation|Children|Comedy|Fantasy  \n",
            "2  Adventure|Animation|Children|Comedy|Fantasy  \n",
            "3  Adventure|Animation|Children|Comedy|Fantasy  \n",
            "4  Adventure|Animation|Children|Comedy|Fantasy  \n"
          ]
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \\\n",
              "userId                                                        \n",
              "1              0.0                                      0.0   \n",
              "2              0.0                                      0.0   \n",
              "3              0.0                                      0.0   \n",
              "4              0.0                                      0.0   \n",
              "5              0.0                                      0.0   \n",
              "\n",
              "title   'Round Midnight (1986)  'Salem's Lot (2004)  \\\n",
              "userId                                                \n",
              "1                          0.0                  0.0   \n",
              "2                          0.0                  0.0   \n",
              "3                          0.0                  0.0   \n",
              "4                          0.0                  0.0   \n",
              "5                          0.0                  0.0   \n",
              "\n",
              "title   'Til There Was You (1997)  'Tis the Season for Love (2015)  \\\n",
              "userId                                                               \n",
              "1                             0.0                              0.0   \n",
              "2                             0.0                              0.0   \n",
              "3                             0.0                              0.0   \n",
              "4                             0.0                              0.0   \n",
              "5                             0.0                              0.0   \n",
              "\n",
              "title   'burbs, The (1989)  'night Mother (1986)  (500) Days of Summer (2009)  \\\n",
              "userId                                                                          \n",
              "1                      0.0                   0.0                          0.0   \n",
              "2                      0.0                   0.0                          0.0   \n",
              "3                      0.0                   0.0                          0.0   \n",
              "4                      0.0                   0.0                          0.0   \n",
              "5                      0.0                   0.0                          0.0   \n",
              "\n",
              "title   *batteries not included (1987)  ...  Zulu (2013)  [REC] (2007)  \\\n",
              "userId                                  ...                              \n",
              "1                                  0.0  ...          0.0           0.0   \n",
              "2                                  0.0  ...          0.0           0.0   \n",
              "3                                  0.0  ...          0.0           0.0   \n",
              "4                                  0.0  ...          0.0           0.0   \n",
              "5                                  0.0  ...          0.0           0.0   \n",
              "\n",
              "title   [REC]² (2009)  [REC]³ 3 Génesis (2012)  \\\n",
              "userId                                           \n",
              "1                 0.0                      0.0   \n",
              "2                 0.0                      0.0   \n",
              "3                 0.0                      0.0   \n",
              "4                 0.0                      0.0   \n",
              "5                 0.0                      0.0   \n",
              "\n",
              "title   anohana: The Flower We Saw That Day - The Movie (2013)  \\\n",
              "userId                                                           \n",
              "1                                                     0.0        \n",
              "2                                                     0.0        \n",
              "3                                                     0.0        \n",
              "4                                                     0.0        \n",
              "5                                                     0.0        \n",
              "\n",
              "title   eXistenZ (1999)  xXx (2002)  xXx: State of the Union (2005)  \\\n",
              "userId                                                                \n",
              "1                   0.0         0.0                             0.0   \n",
              "2                   0.0         0.0                             0.0   \n",
              "3                   0.0         0.0                             0.0   \n",
              "4                   0.0         0.0                             0.0   \n",
              "5                   0.0         0.0                             0.0   \n",
              "\n",
              "title   ¡Three Amigos! (1986)  À nous la liberté (Freedom for Us) (1931)  \n",
              "userId                                                                    \n",
              "1                         4.0                                        0.0  \n",
              "2                         0.0                                        0.0  \n",
              "3                         0.0                                        0.0  \n",
              "4                         0.0                                        0.0  \n",
              "5                         0.0                                        0.0  \n",
              "\n",
              "[5 rows x 9719 columns]"
            ],
            "text/html": [
              "\n",
              "  <div id=\"df-f5472445-69bb-45cb-abf1-8dbd4afdcac6\" class=\"colab-df-container\">\n",
              "    <div>\n",
              "<style scoped>\n",
              "    .dataframe tbody tr th:only-of-type {\n",
              "        vertical-align: middle;\n",
              "    }\n",
              "\n",
              "    .dataframe tbody tr th {\n",
              "        vertical-align: top;\n",
              "    }\n",
              "\n",
              "    .dataframe thead th {\n",
              "        text-align: right;\n",
              "    }\n",
              "</style>\n",
              "<table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              "    <tr style=\"text-align: right;\">\n",
              "      <th>title</th>\n",
              "      <th>'71 (2014)</th>\n",
              "      <th>'Hellboy': The Seeds of Creation (2004)</th>\n",
              "      <th>'Round Midnight (1986)</th>\n",
              "      <th>'Salem's Lot (2004)</th>\n",
              "      <th>'Til There Was You (1997)</th>\n",
              "      <th>'Tis the Season for Love (2015)</th>\n",
              "      <th>'burbs, The (1989)</th>\n",
              "      <th>'night Mother (1986)</th>\n",
              "      <th>(500) Days of Summer (2009)</th>\n",
              "      <th>*batteries not included (1987)</th>\n",
              "      <th>...</th>\n",
              "      <th>Zulu (2013)</th>\n",
              "      <th>[REC] (2007)</th>\n",
              "      <th>[REC]² (2009)</th>\n",
              "      <th>[REC]³ 3 Génesis (2012)</th>\n",
              "      <th>anohana: The Flower We Saw That Day - The Movie (2013)</th>\n",
              "      <th>eXistenZ (1999)</th>\n",
              "      <th>xXx (2002)</th>\n",
              "      <th>xXx: State of the Union (2005)</th>\n",
              "      <th>¡Three Amigos! (1986)</th>\n",
              "      <th>À nous la liberté (Freedom for Us) (1931)</th>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>userId</th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <th>1</th>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>...</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>4.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2</th>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>...</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>3</th>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>...</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>4</th>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>...</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>5</th>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>...</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table>\n",
              "<p>5 rows × 9719 columns</p>\n",
              "</div>\n",
              "    <div class=\"colab-df-buttons\">\n",
              "\n",
              "  <div class=\"colab-df-container\">\n",
              "    <button class=\"colab-df-convert\" onclick=\"convertToInteractive('df-f5472445-69bb-45cb-abf1-8dbd4afdcac6')\"\n",
              "            title=\"Convert this dataframe to an interactive table.\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "  <svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\" viewBox=\"0 -960 960 960\">\n",
              "    <path d=\"M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z\"/>\n",
              "  </svg>\n",
              "    </button>\n",
              "\n",
              "  <style>\n",
              "    .colab-df-container {\n",
              "      display:flex;\n",
              "      gap: 12px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert {\n",
              "      background-color: #E8F0FE;\n",
              "      border: none;\n",
              "      border-radius: 50%;\n",
              "      cursor: pointer;\n",
              "      display: none;\n",
              "      fill: #1967D2;\n",
              "      height: 32px;\n",
              "      padding: 0 0 0 0;\n",
              "      width: 32px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert:hover {\n",
              "      background-color: #E2EBFA;\n",
              "      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "      fill: #174EA6;\n",
              "    }\n",
              "\n",
              "    .colab-df-buttons div {\n",
              "      margin-bottom: 4px;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert {\n",
              "      background-color: #3B4455;\n",
              "      fill: #D2E3FC;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert:hover {\n",
              "      background-color: #434B5C;\n",
              "      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);\n",
              "      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));\n",
              "      fill: #FFFFFF;\n",
              "    }\n",
              "  </style>\n",
              "\n",
              "    <script>\n",
              "      const buttonEl =\n",
              "        document.querySelector('#df-f5472445-69bb-45cb-abf1-8dbd4afdcac6 button.colab-df-convert');\n",
              "      buttonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "\n",
              "      async function convertToInteractive(key) {\n",
              "        const element = document.querySelector('#df-f5472445-69bb-45cb-abf1-8dbd4afdcac6');\n",
              "        const dataTable =\n",
              "          await google.colab.kernel.invokeFunction('convertToInteractive',\n",
              "                                                    [key], {});\n",
              "        if (!dataTable) return;\n",
              "\n",
              "        const docLinkHtml = 'Like what you see? Visit the ' +\n",
              "          '<a target=\"_blank\" href=https://colab.research.google.com/notebooks/data_table.ipynb>data table notebook</a>'\n",
              "          + ' to learn more about interactive tables.';\n",
              "        element.innerHTML = '';\n",
              "        dataTable['output_type'] = 'display_data';\n",
              "        await google.colab.output.renderOutput(dataTable, element);\n",
              "        const docLink = document.createElement('div');\n",
              "        docLink.innerHTML = docLinkHtml;\n",
              "        element.appendChild(docLink);\n",
              "      }\n",
              "    </script>\n",
              "  </div>\n",
              "\n",
              "\n",
              "<div id=\"df-58033cf1-3a29-48c3-9934-787aed4da0ae\">\n",
              "  <button class=\"colab-df-quickchart\" onclick=\"quickchart('df-58033cf1-3a29-48c3-9934-787aed4da0ae')\"\n",
              "            title=\"Suggest charts\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "<svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\"viewBox=\"0 0 24 24\"\n",
              "     width=\"24px\">\n",
              "    <g>\n",
              "        <path d=\"M19 3H5c-1.1 0-2 .9-2 2v14c0 1.1.9 2 2 2h14c1.1 0 2-.9 2-2V5c0-1.1-.9-2-2-2zM9 17H7v-7h2v7zm4 0h-2V7h2v10zm4 0h-2v-4h2v4z\"/>\n",
              "    </g>\n",
              "</svg>\n",
              "  </button>\n",
              "\n",
              "<style>\n",
              "  .colab-df-quickchart {\n",
              "      --bg-color: #E8F0FE;\n",
              "      --fill-color: #1967D2;\n",
              "      --hover-bg-color: #E2EBFA;\n",
              "      --hover-fill-color: #174EA6;\n",
              "      --disabled-fill-color: #AAA;\n",
              "      --disabled-bg-color: #DDD;\n",
              "  }\n",
              "\n",
              "  [theme=dark] .colab-df-quickchart {\n",
              "      --bg-color: #3B4455;\n",
              "      --fill-color: #D2E3FC;\n",
              "      --hover-bg-color: #434B5C;\n",
              "      --hover-fill-color: #FFFFFF;\n",
              "      --disabled-bg-color: #3B4455;\n",
              "      --disabled-fill-color: #666;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart {\n",
              "    background-color: var(--bg-color);\n",
              "    border: none;\n",
              "    border-radius: 50%;\n",
              "    cursor: pointer;\n",
              "    display: none;\n",
              "    fill: var(--fill-color);\n",
              "    height: 32px;\n",
              "    padding: 0;\n",
              "    width: 32px;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart:hover {\n",
              "    background-color: var(--hover-bg-color);\n",
              "    box-shadow: 0 1px 2px rgba(60, 64, 67, 0.3), 0 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "    fill: var(--button-hover-fill-color);\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart-complete:disabled,\n",
              "  .colab-df-quickchart-complete:disabled:hover {\n",
              "    background-color: var(--disabled-bg-color);\n",
              "    fill: var(--disabled-fill-color);\n",
              "    box-shadow: none;\n",
              "  }\n",
              "\n",
              "  .colab-df-spinner {\n",
              "    border: 2px solid var(--fill-color);\n",
              "    border-color: transparent;\n",
              "    border-bottom-color: var(--fill-color);\n",
              "    animation:\n",
              "      spin 1s steps(1) infinite;\n",
              "  }\n",
              "\n",
              "  @keyframes spin {\n",
              "    0% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "      border-left-color: var(--fill-color);\n",
              "    }\n",
              "    20% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    30% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    40% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    60% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    80% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "    90% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "  }\n",
              "</style>\n",
              "\n",
              "  <script>\n",
              "    async function quickchart(key) {\n",
              "      const quickchartButtonEl =\n",
              "        document.querySelector('#' + key + ' button');\n",
              "      quickchartButtonEl.disabled = true;  // To prevent multiple clicks.\n",
              "      quickchartButtonEl.classList.add('colab-df-spinner');\n",
              "      try {\n",
              "        const charts = await google.colab.kernel.invokeFunction(\n",
              "            'suggestCharts', [key], {});\n",
              "      } catch (error) {\n",
              "        console.error('Error during call to suggestCharts:', error);\n",
              "      }\n",
              "      quickchartButtonEl.classList.remove('colab-df-spinner');\n",
              "      quickchartButtonEl.classList.add('colab-df-quickchart-complete');\n",
              "    }\n",
              "    (() => {\n",
              "      let quickchartButtonEl =\n",
              "        document.querySelector('#df-58033cf1-3a29-48c3-9934-787aed4da0ae button');\n",
              "      quickchartButtonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "    })();\n",
              "  </script>\n",
              "</div>\n",
              "\n",
              "    </div>\n",
              "  </div>\n"
            ],
            "application/vnd.google.colaboratory.intrinsic+json": {
              "type": "dataframe",
              "variable_name": "pivot_table"
            }
          },
          "metadata": {},
          "execution_count": 7
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "\n",
        "similarity_matrix = cosine_similarity(pivot_table.T)\n",
        "\n",
        "\n",
        "similarity_df = pd.DataFrame(similarity_matrix, index=pivot_table.columns, columns=pivot_table.columns)\n",
        "\n",
        "\n",
        "print(similarity_df.head())\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "V6k5nUpQ6MvG",
        "outputId": "718ef853-162a-4253-c82d-c449d74865d7"
      },
      "execution_count": 9,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "title                                    '71 (2014)  \\\n",
            "title                                                 \n",
            "'71 (2014)                                      1.0   \n",
            "'Hellboy': The Seeds of Creation (2004)         0.0   \n",
            "'Round Midnight (1986)                          0.0   \n",
            "'Salem's Lot (2004)                             0.0   \n",
            "'Til There Was You (1997)                       0.0   \n",
            "\n",
            "title                                    'Hellboy': The Seeds of Creation (2004)  \\\n",
            "title                                                                              \n",
            "'71 (2014)                                                              0.000000   \n",
            "'Hellboy': The Seeds of Creation (2004)                                 1.000000   \n",
            "'Round Midnight (1986)                                                  0.707107   \n",
            "'Salem's Lot (2004)                                                     0.000000   \n",
            "'Til There Was You (1997)                                               0.000000   \n",
            "\n",
            "title                                    'Round Midnight (1986)  \\\n",
            "title                                                             \n",
            "'71 (2014)                                             0.000000   \n",
            "'Hellboy': The Seeds of Creation (2004)                0.707107   \n",
            "'Round Midnight (1986)                                 1.000000   \n",
            "'Salem's Lot (2004)                                    0.000000   \n",
            "'Til There Was You (1997)                              0.000000   \n",
            "\n",
            "title                                    'Salem's Lot (2004)  \\\n",
            "title                                                          \n",
            "'71 (2014)                                          0.000000   \n",
            "'Hellboy': The Seeds of Creation (2004)             0.000000   \n",
            "'Round Midnight (1986)                              0.000000   \n",
            "'Salem's Lot (2004)                                 1.000000   \n",
            "'Til There Was You (1997)                           0.857493   \n",
            "\n",
            "title                                    'Til There Was You (1997)  \\\n",
            "title                                                                \n",
            "'71 (2014)                                                0.000000   \n",
            "'Hellboy': The Seeds of Creation (2004)                   0.000000   \n",
            "'Round Midnight (1986)                                    0.000000   \n",
            "'Salem's Lot (2004)                                       0.857493   \n",
            "'Til There Was You (1997)                                 1.000000   \n",
            "\n",
            "title                                    'Tis the Season for Love (2015)  \\\n",
            "title                                                                      \n",
            "'71 (2014)                                                           0.0   \n",
            "'Hellboy': The Seeds of Creation (2004)                              0.0   \n",
            "'Round Midnight (1986)                                               0.0   \n",
            "'Salem's Lot (2004)                                                  0.0   \n",
            "'Til There Was You (1997)                                            0.0   \n",
            "\n",
            "title                                    'burbs, The (1989)  \\\n",
            "title                                                         \n",
            "'71 (2014)                                         0.000000   \n",
            "'Hellboy': The Seeds of Creation (2004)            0.000000   \n",
            "'Round Midnight (1986)                             0.176777   \n",
            "'Salem's Lot (2004)                                0.000000   \n",
            "'Til There Was You (1997)                          0.000000   \n",
            "\n",
            "title                                    'night Mother (1986)  \\\n",
            "title                                                           \n",
            "'71 (2014)                                                0.0   \n",
            "'Hellboy': The Seeds of Creation (2004)                   0.0   \n",
            "'Round Midnight (1986)                                    0.0   \n",
            "'Salem's Lot (2004)                                       0.0   \n",
            "'Til There Was You (1997)                                 0.0   \n",
            "\n",
            "title                                    (500) Days of Summer (2009)  \\\n",
            "title                                                                  \n",
            "'71 (2014)                                                  0.141653   \n",
            "'Hellboy': The Seeds of Creation (2004)                     0.000000   \n",
            "'Round Midnight (1986)                                      0.000000   \n",
            "'Salem's Lot (2004)                                         0.000000   \n",
            "'Til There Was You (1997)                                   0.000000   \n",
            "\n",
            "title                                    *batteries not included (1987)  ...  \\\n",
            "title                                                                    ...   \n",
            "'71 (2014)                                                          0.0  ...   \n",
            "'Hellboy': The Seeds of Creation (2004)                             0.0  ...   \n",
            "'Round Midnight (1986)                                              0.0  ...   \n",
            "'Salem's Lot (2004)                                                 0.0  ...   \n",
            "'Til There Was You (1997)                                           0.0  ...   \n",
            "\n",
            "title                                    Zulu (2013)  [REC] (2007)  \\\n",
            "title                                                                \n",
            "'71 (2014)                                       0.0      0.342055   \n",
            "'Hellboy': The Seeds of Creation (2004)          0.0      0.000000   \n",
            "'Round Midnight (1986)                           0.0      0.000000   \n",
            "'Salem's Lot (2004)                              0.0      0.000000   \n",
            "'Til There Was You (1997)                        0.0      0.000000   \n",
            "\n",
            "title                                    [REC]² (2009)  \\\n",
            "title                                                    \n",
            "'71 (2014)                                    0.543305   \n",
            "'Hellboy': The Seeds of Creation (2004)       0.000000   \n",
            "'Round Midnight (1986)                        0.000000   \n",
            "'Salem's Lot (2004)                           0.000000   \n",
            "'Til There Was You (1997)                     0.000000   \n",
            "\n",
            "title                                    [REC]³ 3 Génesis (2012)  \\\n",
            "title                                                              \n",
            "'71 (2014)                                              0.707107   \n",
            "'Hellboy': The Seeds of Creation (2004)                 0.000000   \n",
            "'Round Midnight (1986)                                  0.000000   \n",
            "'Salem's Lot (2004)                                     0.000000   \n",
            "'Til There Was You (1997)                               0.000000   \n",
            "\n",
            "title                                    anohana: The Flower We Saw That Day - The Movie (2013)  \\\n",
            "title                                                                                             \n",
            "'71 (2014)                                                                             0.0        \n",
            "'Hellboy': The Seeds of Creation (2004)                                                0.0        \n",
            "'Round Midnight (1986)                                                                 0.0        \n",
            "'Salem's Lot (2004)                                                                    0.0        \n",
            "'Til There Was You (1997)                                                              0.0        \n",
            "\n",
            "title                                    eXistenZ (1999)  xXx (2002)  \\\n",
            "title                                                                  \n",
            "'71 (2014)                                           0.0    0.139431   \n",
            "'Hellboy': The Seeds of Creation (2004)              0.0    0.000000   \n",
            "'Round Midnight (1986)                               0.0    0.000000   \n",
            "'Salem's Lot (2004)                                  0.0    0.000000   \n",
            "'Til There Was You (1997)                            0.0    0.000000   \n",
            "\n",
            "title                                    xXx: State of the Union (2005)  \\\n",
            "title                                                                     \n",
            "'71 (2014)                                                     0.327327   \n",
            "'Hellboy': The Seeds of Creation (2004)                        0.000000   \n",
            "'Round Midnight (1986)                                         0.000000   \n",
            "'Salem's Lot (2004)                                            0.000000   \n",
            "'Til There Was You (1997)                                      0.000000   \n",
            "\n",
            "title                                    ¡Three Amigos! (1986)  \\\n",
            "title                                                            \n",
            "'71 (2014)                                                 0.0   \n",
            "'Hellboy': The Seeds of Creation (2004)                    0.0   \n",
            "'Round Midnight (1986)                                     0.0   \n",
            "'Salem's Lot (2004)                                        0.0   \n",
            "'Til There Was You (1997)                                  0.0   \n",
            "\n",
            "title                                    À nous la liberté (Freedom for Us) (1931)  \n",
            "title                                                                               \n",
            "'71 (2014)                                                                     0.0  \n",
            "'Hellboy': The Seeds of Creation (2004)                                        0.0  \n",
            "'Round Midnight (1986)                                                         0.0  \n",
            "'Salem's Lot (2004)                                                            0.0  \n",
            "'Til There Was You (1997)                                                      0.0  \n",
            "\n",
            "[5 rows x 9719 columns]\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "def recommend_movies(movie_title, similarity_df, num_recommendations=5):\n",
        "\n",
        "    sim_scores = similarity_df[movie_title]\n",
        "\n",
        "\n",
        "    sim_scores = sim_scores.sort_values(ascending=False)\n",
        "\n",
        "\n",
        "    recommendations = sim_scores.iloc[1:num_recommendations+1].index\n",
        "\n",
        "    return recommendations\n"
      ],
      "metadata": {
        "id": "Ju_EWuN46WZ1"
      },
      "execution_count": 11,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "\n",
        "movie_title = 'Toy Story (1995)'\n",
        "recommendations = recommend_movies(movie_title, similarity_df, num_recommendations=5)\n",
        "\n",
        "print(f\"Recommendations for '{movie_title}':\")\n",
        "for movie in recommendations:\n",
        "    print(movie)\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "KYdHSfB-6dQ0",
        "outputId": "8af4ae42-bc0b-45cc-84ed-b50e9d30659a"
      },
      "execution_count": 17,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Recommendations for 'Toy Story (1995)':\n",
            "Toy Story 2 (1999)\n",
            "Jurassic Park (1993)\n",
            "Independence Day (a.k.a. ID4) (1996)\n",
            "Star Wars: Episode IV - A New Hope (1977)\n",
            "Forrest Gump (1994)\n"
          ]
        }
      ]
    }
  ]
}

{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'provenance': []},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'},
  'language_info': {'name': 'python'}},
 'cells': [{'cell_type': 'code',
   'execution_count': 2,
   'metadata': {'id': 'KKBEvZqx4vvY'},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'from sklearn.metrics.pairwise import cosine_similarity\n',
    'from sklearn.feature_extraction.text import TfidfVectorizer\n',
    'from scipy.sparse.linalg import svds']},
  {'cell_type': 'code',
   'source': ['\n',
    '!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip\n',
    '!unzip -n ml-latest-small.zip\n',
    '\n',
    '\n',
    "movies = pd.read_csv('/content/ml-latest-small/movies.csv')\n",
    "ratings = pd.read_csv('/content/ml-latest-small/ratings.csv')\n",
    '\n',
    '\n',
    'print(movies.head())\n',
   